# Day 2 – Tutorial 02: Coevolutionary analysis of protein sequences using pyDCA

###Theoretical aspects

The conservation of the three-dimensional structure among homologous proteins imposes constrains on sequence variability. Even when the sequence identity between homologous members from a given protein family can be even below the so-called twilight zone (20-30% sequence identity), their structures often exhibit **interactions between residue pairs that, when mapped onto their protein sequence, occupy equivalent positions among all proteins**. While the chemical nature of the interaction between these residue pairs can vary from one protein to another, the physical contact between these residues is **constrained in the final structure**.

Based on this, it was suggested that the **statistical analysis of correlated amino acid substitution patterns at different sequence positions of a protein family** could be exploited to **infer spatial contacts within the tertiary protein structure**. The hypothesis behind it is fairly simple: if two residues are interacting in a protein structure and a destabilizing mutations occurs in one of the residues, **a compensatory mutation at an specific residue position in a sequence preserves the stability of the protein architecture and function (FIG 1).**  



<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/sbmdca_01.png'/ width=700>
<figcaption><b>FIGURE 1. </b>Schematic representation of the residue pair contacts that can be inferred from the coevolutionary analysis of linear protein sequences.<br>The left side of the image represents an alignment of multiple sequences of the same protein from many organisms. The boxed columns indicate two positions in this alignment whose residues are always complementary to each other, in spite of the mutations occurring in each position. This pattern suggest that these two positions likely form a physical interaction crucial for the stability of the protein structure, as shown on the right side.<br>
Cartoon by Sergey Ovchinikov, <i><a href="http://site.solab.org/home">http://site.solab.org/home</a></i></figcaption></center>
</figure>

Thanks to the explosion of genomic sequencing, which has enabled the data deposition of **thousands of protein sequences** in publicly available databases, these statistical signatures left by protein evolution can be ascertained from sequence information alone to predict which residue pairs should be in spatial proximity in the native functional protein fold.

##Experimental Overview

In this tutorial, we will exemplify how we can **infer native contacts from the coevolutionary analysis of protein sequence information using Direct Coupling Analysis (DCA)**.

As a continuation of tutorial 1, we will particularly focus on comparing the inferred and experimental residue pair interactions of the **Fox family of transcription factors** ([**Pfam family PF00250**](https://pfam.xfam.org/family/PF00250)).

For the coevolutionary analysis of linear protein sequences, we will employ the **DCA** method. The DCA scores deliver quantitative information about the existence of physical contacts in a three-dimensional structure of a biomolecule, with top-scoring pairs accurately predicting native contacts that can be observed in experimentally solved protein structures (and also RNA!).

The following tutorial is based on the pyDCA by Mehari B. Zerihun (available in [this GitHub](https://github.com/KIT-MBS/pydca)), which is associated with the publication:
  -  Zerihun, M. B., Pucci, F., Peter, E. K., & Schug, A. (2020). pydca v1. 0: a comprehensive software for Direct Coupling Analysis of RNA and Protein Sequences. [*Bioinformatics, 36*(7), 2264-2265](https://doi.org/10.1093/bioinformatics/btz892)

#Part 0. Downloading and Installing the required software

We first must install several pieces of software to perform this tutorial. Namely:

- **biopython** for manipulation of the PDB files
- **py3Dmol** for visualization of the protein structure.
- **HMMER** for aligning multiple protein sequences using profile Hidden Markov Model (HMM) of position-specific conservation.
- **pyDCA** for predicting native contacts through Direct Coupling Analysis.

**WARNING⚠️:** pyDCA requires more recent version of several python modules, and thus a **runtime restart** will be requested. If you are requested to perform a runtime restart, please do!


1. We will first install **HMMER** from the Advanced Package Tool (APT)

In [ ]:
!apt-get install hmmer

2. We will then install **py3Dmol**, **pydca** and **biopython** simultaneously as they are all available through `pip`

In [ ]:
!pip install pydca biopython py3Dmol

Once these software installation processes are completed, we are ready to perform our experiments

#Part I – Examine the three-dimensional structure of human FoxP1

We will first download and visualize the solved structure of the DNA-binding domain of human FoxP1 (PDB 2kiu) through **biopython** and **py3Dmol**.

1. Download the structure from the PDB using biopython:

In [ ]:
#Importing your PDB file using biopython
import os
from Bio.PDB import *

#Here, we add a unique or multiple PDB accession IDs
pdbid = ['2kiu']

#We will treat the IDs as a list to download all PDBs
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")
print("DONE!")

💡 **HINT**: When working with protein structures solved by X-ray crystallography, the structure is often accompanied by crystallographic water molecules, and could also contains substrates or ions. We are running this command below although it is not required in this specific case (2kiu is a solution structure from NMR data)

2. Clean up the protein structure from crystallographic waters and other non-protein atoms with biopython:

In [ ]:
#Here we set up a parser for our PDB
parser = PDBParser()
io=PDBIO()
structure = parser.get_structure('A', '2kiu.pdb')
#Now we remove hydrogens, waters and ligands using Dice
#and save the cleaned-up structure with a different filename
io.set_structure(structure)
sel = Dice.ChainSelector('A', 1, 10000)
io.save("2kiu_A.pdb", sel)
print("Your PDB was processed. Only the protein heavy atoms have been kept")

3. Lastly, we will visualize the three-dimensional structure of the DNA-binding domain of human FoxP1 using py3Dmol.

**NOTE❗️** For other color schemes, check the available ones at the [3Dmol](https://3dmol.csb.pitt.edu/doc/types.html#ColorschemeSpec) website.

In [ ]:
import py3Dmol
#First we assign the py3Dmol.view as view
view=py3Dmol.view()
#The following lines are used to add the addModel class
#to read the PDB files
view.addModel(open('2kiu_A.pdb', 'r').read(),'pdb')
#Here we set the background color as white
view.setBackgroundColor('white')
#Here we set the visualization style and color
view.setStyle({'chain':'A'},{'cartoon': {'colorscheme':'ssJmol'}})
#You can activate the labels for each residue if you want
#or comment them with a '#' at the beggining of each line if you do not want to
view.addResLabels({'resi':'1'},{'fontColor':'white','fontOpacity':1,'showBackground':'true'})
view.addResLabels({'resi':'87'},{'fontColor':'white','fontOpacity':1,'showBackground':'true'})

#Here we center the molecule for its visualization
view.zoomTo()
#And we finally visualize the structures using the command below
view.show()

#Part II – Generate a multiple sequence alignment (MSA) using HMMER 

As illustrated in FIG 1, a multiple sequence alignment (MSA) of many sequences of a protein family is required to infer physical interactions. A natural question that arises is where to obtain such sequences.

As previously indicated, human FoxP1 is a member of the **Fox family of transcription factors** in the **Pfam** database [**PF00250**](https://pfam.xfam.org/family/PF00250). The benefit of Pfam is that it contains **seed alignments**, i.e. small MSA based on a subset of known sequences that are used to **construct profile HMMs** to search for other protein homologs or to generate a larger MSA, as well as **readily available MSAs of identified sequences using these HMM profiles**.

We will use these seed alignments and known Hemerythrin family protein sequences as inputs for generating a HMM-based MSA using HMMER.

1. We will first create a folder (**_dca_analysis_**) in which we will save all of the input and output files from our coevolutionary analysis.

In [ ]:
#Let's make a folder for the coevolutionary analysis
#We need to import the os and path library
import os
from pathlib import Path 

#Then, we define the path of the folder we want to create.
#Notice that the HOME folder for a hosted runtime in colab is /content/
dcapath = Path("/content/dca_analysis/")

#Now, we create the folder using the os.mkdir() command
#The 'if' conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(dcapath):
  print("path already exists")
if not os.path.exists(dcapath):
  os.mkdir(dcapath)
  print("path was succesfully created")

#We finally move onto this new folder
os.chdir(dcapath)

2. We will now download the sequence of the DNA-binding domain of human FoxP1 (PDB 2kiu) in FASTA format using biopython.

**NOTE❗️** Given that PDB files can contain more than one polypeptide chain, it is required to specify the chain ID from which the protein sequence will be extracted. In our case, 2MHR only contains a single chain A, thus our accession ID is *2MHR_A*

In [ ]:
import os
from pathlib import Path 
from Bio import SeqIO, Entrez
from Bio.SeqRecord import SeqRecord
#Indicate the accession ID of the protein sequence to download.
seqlist = ['2KIU_A']
for n in seqlist:
  #Setting up your email to be able to use Entrez
  Entrez.email = 'your.email@mail.com'
  #Here, we set up a temporary handle with our downloaded sequence
  #in fasta format
  temp = Entrez.efetch(db="sequences",rettype="fasta",id=n)
  #Creating a fasta file to write our downloaded sequence
  seq_out = open("2kiu_A.fasta",'w')
  #Reading the sequence information as a string in fasta format
  seq = SeqIO.read(temp, format="fasta")
  print("The downloaded protein sequence is:\n" + seq.seq + "\n")
  SeqIO.write(seq,seq_out,"fasta")
  #Closing both the temp handle and the FASTA file
  temp.close()
  seq_out.close()

3. Then, we will go to the [**Pfam** database](https://pfam.xfam.org) and search for the appropriate protein family. You can easily do this by inputting **_Forkhead Domain_** or **_PF00250_** in the **Search by Text** option of the search box. Once here, we will obtain two different files. The first one is the **seed alignment**.

  - Go to the **_Alignments_ → _Available alignments_** menu and **download the seed alignment**, which is used for constructing a profile HMM.
  - Then, upload your file into the **dca_analysis** folder by dragging it and dropping it.

  Here, we simplify this exercise by directly downloading the seed alignment in Google Colab as a text file (*PF00250.alignment.seed*).
  
4. Next, we will use `hmmbuild` from HMMER to generate our profile HMM.

In [ ]:
#Downloading the seed file for PF00250
!wget https://raw.githubusercontent.com/pb3lab/workshops/main/backups/saocarlos2022/PF00250.alignment.seed

#Building the hmm profile from the seed file for PF00250
!hmmbuild PF00250.cm PF00250.alignment.seed

5. The second file is **a set of sequences from one of the available databases** in STOCKHOLM format. While the higher the number of sequences the better, we **highly recommend to download the sequences from RP15, RP35 or RP55** due to our time constraints:

    - Go to the **_Alignments_ → _Available alignments_** menu and **download the available alignment**, which is used for constructing a profile HMM.
    - Then, upload your file into the **dca_analysis** folder by dragging it and dropping it.
    - Since we will add our target sequence to this sequence set in order to have our protein represented in our multiple sequence alignment, we have to change the file format from STOCKHOLM to FASTA and also to delete the gapsusing **biopython**, as shown below.

**⚠️WARNING:** We have recently checked that the Pfam website is a little bit slow. Hence, we have uploaded the sequences for PF00250 onto our GitHub. Please choose between RP15, RP35, RP55, RP75 or UNIPROT as shown below:

In [ ]:
#Downloading the rp15 alignment file for PF00250
#The db variable can be altered to rp15, rp35, rp55, rp75, or uniprot
db = 'rp55'

!wget https://raw.githubusercontent.com/pb3lab/workshops/main/backups/saocarlos2022/PF00250.alignment.{db}
!mv PF00250.alignment.{db} PF00250.alignment.stockholm

#Generating an ungapped FASTA file with all sequences from PFAM
from Bio.Seq import Seq
from Bio import SeqIO
with open("PF00250.seqs.fasta", "w") as o:
    for record in SeqIO.parse("PF00250.alignment.stockholm", "stockholm"):
        record.seq = record.seq.ungap("-")
        SeqIO.write(record, o, "fasta-2line")

4. Then, we will add our target sequence to our sequence set and use the profile HMM to generate a MSA using `hmmalign` from HMMER. The resulting MSA will be stored as **_PF00250_2kiu_aligned.afa_**

In [ ]:
!cat PF00250.seqs.fasta 2kiu_A.fasta > PF00250_2kiu.fasta
!hmmalign -o PF00250_2kiu_aligned.afa --outformat AFA PF00250.cm PF00250_2kiu.fasta

#Part III – Infer physical interactions between residue pairs in an MSA through coevolutionary analysis on pyDCA

The resulting MSA will be our input for our coevolutionary analysis using the **mean-field approximation of DCA (mfDCA)**, one of the two available approaches (the other being pseudo-maximum likelihood or plmDCA) in pyDCA, with mfDCA being faster to compute. 

pyDCA is a python implementation of DCA that not only infers physical contacts from the MSA, but also ascertains the accuracy of our prediction by establishing the number of true positives when compared to the contacts observed in a solved protein structure.

Briefly, DCA will allow for disentangling direct contributions to correlations (resulting from native contacts) from indirect contributions (mediated through chains of native contacts). Given that our intention is to predict the structure of *T. hennahi* myohemerythrin based on sequence information alone, we will use its protein sequence as reference for the prediction of residue-pair contacts.

**NOTE❗️** While the statistical physics algorithms available in pyDCA are outside ths scope of this tutorial, a deep dive into mfDCA and plmDCA can be found in:
- Ekeberg, M., Lövkvist, C., Lan, Y., Weigt, M., & Aurell, E. (2013). Improved contact prediction in proteins: using pseudolikelihoods to infer Potts models. [*Physical Review E, 87(1), 012707.*](https://doi.org/10.1103/physreve.87.012707)

1. We will first import all of the required modules for pyDCA

In [ ]:
from pydca.plmdca import plmdca
from pydca.meanfield_dca import meanfield_dca
from pydca.sequence_backmapper import sequence_backmapper
from pydca.msa_trimmer import msa_trimmer
from pydca.contact_visualizer import contact_visualizer
from pydca.dca_utilities import dca_utilities

2. Then, we will trim our MSA file based on the lenght of the target sequence, which in this case corresponds to the sequence of the DNA-binding domain of human FoxP1. The input MSA is defined in the `prot_msa_file` path, whereas our target sequence is defined in the `prot_refseq_file`. The trimmed MSA data to an output file (**_MSA_PF00250_2kiu_trimmed.fa_**) in FASTA format.

In [ ]:
#Paths to our MSA and reference sequence files

prot_msa_file = 'PF00250_2kiu_aligned.afa'
prot_refseq_file = '2kiu_A.fasta'

#Create MSATrimmer instance 
trimmer = msa_trimmer.MSATrimmer(
    prot_msa_file, biomolecule='protein', 
    refseq_file=prot_refseq_file,
)

trimmed_data = trimmer.get_msa_trimmed_by_refseq(remove_all_gaps=True)

#write trimmed MSA to file in FASTA format
trimmed_data_outfile = 'MSA_PF00250_2kiu_trimmed.fa'
with open(trimmed_data_outfile, 'w') as fh:
    for seqid, seq in trimmed_data:
        fh.write('>{}\n{}\n'.format(seqid, seq))

3. Once done, we will compute our DCA scores using the **mean-field approach (mfDCA)**. For this analysis, we first create a mfDCA instance `mfdca_inst` to analyze our trimmed MSA. We also provide a series of optional parameters:
- A pseudocount, which is basically adding extra “pseudo” observations to the real data in order to cure singularities caused by strong correlations in undersampled data, using the `pseudocount` parameter. This value is typically set to one-third (0.3) to one-half (0.5) of the total numer of sequences in the MSA. 
- A maximum sequence identity percentage for all sequences in the MSA, using the `seqid` parameter. This value is typically set to 80% (0.8) sequence identity.
- The number of threads, which is set to 2, the maximum for Google Colab.

  After the fields and couplings are obtained, the DCA scores are computed from the Frobenius norm of the couplings between sites $i$ and $j$ by calling the `compute_sorted_FN_APC()` method on `mfdca_inst`. This action returns the average product corrected (APC) DCA scores.


In [ ]:
# Compute DCA scores using the mean-field algorithm
mfdca_inst = meanfield_dca.MeanFieldDCA(
    trimmed_data_outfile,
    'protein',
    pseudocount = 0.5,
    seqid = 0.8,
)

# Compute average product corrected Frobenius norm of the couplings
mfdca_FN_APC = mfdca_inst.compute_sorted_FN_APC()

4. Once these mfDCA calculations are finished, we can print them out on screen. Here, we will use python to print the top 10 DCA pairs.

In [ ]:
for site_pair, score in mfdca_FN_APC[:10]:
    print(site_pair, score)

5. A more meaningful way to determine how good are our predictions of physical contacts is to visually and statistically compare them against experimental data.

  For this purpose, we will generate a `DCAVisualizer` to compare our inferred contact map against the experimental one derived from the protein structure of the DNA-binding domain of human FoxP1.
  
  This visualizer takes the type of biomolecule (`'protein'`), our reference sequence file (`refseq_file`) and our previously generated list of sorted DCA scores (`sorted_dca_scores`) as input to generate a contact map based on sequence information alone.

  When structural information is available, it also takes a PDB ID (`'2kiu_A'`) and chain (`'A'`) as input parameters, as well as specified linear (i.e. sequence separation between residue pairs, `linear_dist`) and contact distances (`contact_dist`, in Å).

In [ ]:
mfdca_visualizer = contact_visualizer.DCAVisualizer('protein', 'A', '../2kiu_A.pdb',
    refseq_file = prot_refseq_file,
    sorted_dca_scores = mfdca_FN_APC,
    linear_dist = 4,
    contact_dist = 8.0,
)

6. We will now plot both contact maps, with the structure-based contacts on the upper left triangle, and the DCA-based contacts on the lower right triangle. Correctly predicted contacts are shown in green, whereas false positives are shown in red. Also note that the number of DCA contacts is equivalent to $L$, where $L$ is the lenght (i.e. number of columns) of the trimmed alignment.

In [ ]:
contact_map_data = mfdca_visualizer.plot_contact_map()

7. Lastly, we will determine the accuracy of our coevolutionary analysis by looking at its true positive (TP) rate per rank. The TP rate per rank is the number of correctly predicted contacts per rank of the predicted pairs divided by all predictions at that rank.

  In the resulting plot, the blue line will correspond to the TP rates for the predicted contacts, and the orange line to the theoretically maximum possible true positive rate for the contacts obtained from the experimental structure.

In [ ]:
tp_rate_data = mfdca_visualizer.plot_true_positive_rates()

📚**HOMEWORK**: Check what happens with the number of TP contacts predicted when increasing the number of sequences in the alignment (i.e. changing from RP15 to RP35, RP55 or even RP75).

# Part IV - Backing up your files

1. If you want to download your produced files, execute the code below. A compressed .tar.gz file will be generated and automatically downloaded into your computer (unless you have an ad-blocker, for which you will have to manually download it).

In [ ]:
#Going back to the /content/ directory first
%cd /content/

In [ ]:
#Compressing all files into a .tar.gz file
!tar -czf D2-tutorial-02.tar.gz *

In [ ]:
from google.colab import files
files.download("/content/D2-tutorial-02.tar.gz")

2. Alternatively, you can transfer the files directly to your Google Drive as shown below:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil
from pathlib import Path 
backup = Path("/content/drive/MyDrive/saocarlos2022/")
if os.path.exists(backup):
  print("Sao Carlos Workshop 2022 - Backup folder already exists")
if not os.path.exists(backup):
  os.mkdir(backup)
  print("Sao Carlos Workshop 2022 - Backup folder did not exists and was succesfully created")

#Backing up
shutil.copy(str('/content/D2-tutorial-02.tar.gz'), str(backup/'D2-tutorial-02.tar.gz'))
print("Day 2 - Tutorial 2 files successfully backed up!")